In [1]:
%env TF_DISABLE_MLC=1

env: TF_DISABLE_MLC=1


In [2]:
from mtg.obj.games import Games
from mtg.ml.models import DeckBuilder
from mtg.ml.trainer import Trainer
from mtg.utils.display import print_deck
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
data_file = '/Users/a10889158/personal/mtg/data/game_data_public.KHM.PremierDraft.csv'

In [4]:
%%time
games = Games(data_file,'khm')

/Users/a10889158/miniforge3/envs/personal/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


CPU times: user 9.34 s, sys: 3.57 s, total: 12.9 s
Wall time: 15.1 s


In [5]:
train_decks,test_decks = games.get_decks_for_ml()

In [6]:
land_idxs = games.cards[games.cards['type_line'].apply(lambda x: "land" in x.lower())]['idx'].to_numpy()

In [7]:
model = DeckBuilder(len(games.cards),land_idxs)
model.compile(
    optimizer=tf.optimizers.Adam(lr=0.001),
    loss_function=tf.losses.mean_squared_error,
    reg_coef=1.0,
    constraint_coef=0.1,
)
t = Trainer(train_decks, train_decks, model)
t.train(100, batch_size=64)

Epoch 100/100: 100%|██████████| 718/718 [00:09<00:00, 75.55Batch/s, loss=0.0725]


In [61]:
output = model.compare(test_decks[23], games.cards).fillna(0)

In [62]:
output

,name,real deck,fake sideboard,predicted deck
0,mountain,9,0.0,8.0
1,plains,8,0.0,8.0
2,skemfar elderhall,0,1.0,0.0
3,alpine meadow,0,1.0,0.0
4,brinebarrow intruder,0,1.0,0.0
5,usher of the fallen,2,0.0,2.0
6,draugr's helm,0,1.0,0.0
7,goldvein pick,0,1.0,1.0
8,raven wings,0,2.0,2.0
9,axgard cavalry,1,0.0,1.0


In [63]:
#print real deck to show in deckbuilder tool
_ = output.apply(lambda x: print(x['real deck'],x['name'].capitalize()) if x['real deck'] > 0 else None, axis=1)

9 Mountain
8 Plains
2 Usher of the fallen
1 Axgard cavalry
1 Story seeker
1 Vault robber
1 Run amok
1 Magda, brazen outlaw
1 Koll, the forgemaster
1 Bloodline pretender
1 Dwarven hammer
3 Goldmaw champion
1 Tuskeri firewalker
2 Demon bolt
1 Iron verdict
1 Seize the spoils
1 Basalt ravager
1 Showdown of the skalds
1 Provoke the trolls
1 Dwarven reinforcements
1 Warhorn blast


In [64]:
#print fake pool to show in deckbuilder tool
_ = output.apply(lambda x: print(x['real deck'] + int(x['fake sideboard']),x['name']) if x['real deck'] + x['fake sideboard'] > 0 else None, axis=1)

9 mountain
8 plains
1 skemfar elderhall
1 alpine meadow
1 brinebarrow intruder
2 usher of the fallen
1 draugr's helm
1 goldvein pick
2 raven wings
1 axgard cavalry
1 story seeker
1 vault robber
1 skemfar avenger
2 withercrown
1 invasion of the giants
1 run amok
1 magda, brazen outlaw
1 koll, the forgemaster
1 strategic planning
1 bloodline pretender
1 dwarven hammer
3 goldmaw champion
1 tuskeri firewalker
1 gnottvold recluse
1 invoke the divine
2 demon bolt
1 kaya's onslaught
1 iron verdict
1 reidane, god of the worthy // valkmira, protector's shield
1 svella, ice shaper
1 seize the spoils
1 vengeful reaper
1 basalt ravager
1 dogged pursuit
1 showdown of the skalds
1 the trickster-god's heist
1 provoke the trolls
1 draugr necromancer
1 struggle for skemfar
1 dwarven reinforcements
1 warhorn blast
1 doomskar
1 blood on the snow
1 cyclone summoner


In [65]:
#print predicted deck to show in deckbuilder tool
_ = output.apply(lambda x: print(int(x['predicted deck']),x['name']) if x['predicted deck'] > 0 else None, axis=1)

8 mountain
8 plains
2 usher of the fallen
1 goldvein pick
2 raven wings
1 axgard cavalry
1 story seeker
1 vault robber
1 run amok
1 koll, the forgemaster
1 bloodline pretender
1 dwarven hammer
3 goldmaw champion
1 tuskeri firewalker
2 demon bolt
1 kaya's onslaught
1 iron verdict
1 basalt ravager
1 showdown of the skalds
1 provoke the trolls
1 dwarven reinforcements
1 warhorn blast


In [66]:
#show number of cards in each column
output.sum(axis=0)

name              mountainplainsskemfar elderhallalpine meadowbr...
real deck                                                        40
fake sideboard                                                 25.0
predicted deck                                                 41.0
dtype: object

In [14]:
# basics = games.cards[
#     games.cards['type_line'].apply(lambda x: x.lower().startswith("basic land"))
# ]['idx'].to_numpy()
# not_basics = np.asarray(list(set(np.arange(len(games.cards))) - set(basics)))
# sideboard_size = 45 - decks[:,not_basics].sum(1)

In [15]:
# p = decks.sum(0)
# p[basics] = 0
# p = p/p.sum()
# def generate_sideboard(n_cards):
#     size=len(p)
#     sideboard = np.random.choice(np.arange(size),size=n_cards,replace=True,p=p)
#     unique, counts = np.unique(sideboard, return_counts=True)
#     output = np.zeros(size)
#     output[unique] = counts
#     return output